In [1]:
# 1. Core LangChain + integrations
!pip install -U langchain langchain-huggingface -q
!pip install langchain_community langchain_google_genai langchain-astradb -q

# 2. Document loaders (pptx, pdf, text, unstructured)
!pip install python-pptx pypdf unstructured -q

# 3. Clean up old Hugging Face libs (remove conflicts)
!pip uninstall -y sentence-transformers transformers -q

# 4. Reinstall modern Hugging Face + Torch stack
!pip install -U torch transformers sentence-transformers -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.5/300.5 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.0/444.0 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 69.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 24.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2

In [2]:
import os
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
ASTRADB_TOKEN = userdata.get('ASTRADB_TOKEN')

In [3]:
from langchain_huggingface import HuggingFaceEmbeddings

# Best quality HuggingFace embedding model
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2",
    model_kwargs={"device": "cpu"}  # or "cuda" if GPU is available
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
root_dir="/content/"
docs_folder_path = f'{root_dir}/docs/'
os.listdir(docs_folder_path)

['movies_top_ten.txt', 'aaa_games_info.pdf', 'Generative_AI_Roadmap.pptx']

## Data Ingestion

In [5]:
from langchain_community.document_loaders import (
    PyPDFLoader,
    TextLoader,
    UnstructuredPowerPointLoader
)
import os

docs_path = docs_folder_path
documents = []

for file in os.listdir(docs_path):
    file_path = os.path.join(docs_path, file)

    if file.endswith(".pdf"):
        loader = PyPDFLoader(file_path)
    elif file.endswith(".txt"):
        loader = TextLoader(file_path, encoding="utf-8")
    elif file.endswith(".pptx"):
        loader = UnstructuredPowerPointLoader(file_path)
    else:
        continue  # skip unsupported files

    documents.extend(loader.load())

print(f"Loaded {len(documents)} chunks from docs/")
print(documents[0])  # preview first chunk


Loaded 3 chunks from docs/
page_content='Text: Top 10 IMDb Movies

1. The Shawshank Redemption (1994)
Directed by Frank Darabont. Starring Tim Robbins and Morgan Freeman.
A powerful story of hope and friendship inside Shawshank prison. Andy Dufresne, sentenced for a crime he didn’t commit, builds a legacy of resilience and justice.

2. The Godfather (1972)
Directed by Francis Ford Coppola. Starring Marlon Brando and Al Pacino.
An epic saga of the Corleone mafia family, exploring power, loyalty, and betrayal in organized crime.

3. The Dark Knight (2008)
Directed by Christopher Nolan. Starring Christian Bale and Heath Ledger.
A gritty superhero masterpiece where Batman faces his deadliest rival, the Joker, in a moral battle for Gotham City.

4. The Godfather Part II (1974)
Directed by Francis Ford Coppola. Starring Al Pacino and Robert De Niro.
A dual narrative showing the rise of young Vito Corleone and Michael Corleone’s consolidation of power.

5. 12 Angry Men (1957)
Directed by Sidn

Method 2: Using Unstructured Loader (UnstructuredFileLoader)

👉 Handles multiple formats automatically (PDF, TXT, PPTX, DOCX, HTML, etc.) with OCR + layout parsing.
⚠️ Requires Python ≤ 3.11 (not yet Py3.12-ready).

```
from langchain_community.document_loaders import UnstructuredFileLoader
import os

docs_path = "docs/"
documents = []

for file in os.listdir(docs_path):
    file_path = os.path.join(docs_path, file)
    
    if file.lower().endswith((".pdf", ".txt", ".pptx")):
        loader = UnstructuredFileLoader(file_path, mode="elements")  # "elements" = structured chunks
        documents.extend(loader.load())

print(f"Loaded {len(documents)} structured chunks from docs/")
print(documents[0])  # preview first structured chunk
```
Pros: Auto-handles multiple formats, OCR support, keeps structure.

Cons: Heavy dependencies (onnxruntime, tesseract, etc.), not stable in Python 3.12.

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Create a text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,   # max characters per chunk
    chunk_overlap=200, # overlap between chunks to preserve context
    length_function=len,
)

# Apply to all docs
split_documents = text_splitter.split_documents(documents)

print(f"After splitting: {len(split_documents)} chunks")
print(split_documents[0])

After splitting: 16 chunks
page_content='Text: Top 10 IMDb Movies

1. The Shawshank Redemption (1994)
Directed by Frank Darabont. Starring Tim Robbins and Morgan Freeman.
A powerful story of hope and friendship inside Shawshank prison. Andy Dufresne, sentenced for a crime he didn’t commit, builds a legacy of resilience and justice.

2. The Godfather (1972)
Directed by Francis Ford Coppola. Starring Marlon Brando and Al Pacino.
An epic saga of the Corleone mafia family, exploring power, loyalty, and betrayal in organized crime.

3. The Dark Knight (2008)
Directed by Christopher Nolan. Starring Christian Bale and Heath Ledger.
A gritty superhero masterpiece where Batman faces his deadliest rival, the Joker, in a moral battle for Gotham City.

4. The Godfather Part II (1974)
Directed by Francis Ford Coppola. Starring Al Pacino and Robert De Niro.
A dual narrative showing the rise of young Vito Corleone and Michael Corleone’s consolidation of power.' metadata={'source': '/content//docs/mov

In [7]:
from langchain_astradb import AstraDBVectorStore
ASTRADB_ENDPOINT = "https://d8b9caac-52cc-49fc-b343-8c5c0ce2c937-us-east-2.apps.astra.datastax.com"
# ASTRADB_TOKEN
ASTRA_DB_KEYSPACE = "default_keyspace"

In [8]:
vstore = AstraDBVectorStore(
    embedding=embeddings,
    collection_name="multidocs_astra_langchain_collection",
    api_endpoint=ASTRADB_ENDPOINT,
    token=ASTRADB_TOKEN,
    namespace=ASTRA_DB_KEYSPACE,
)

In [9]:
vstore.add_documents(split_documents)

['f5c239fc8b1f49e6988a5a706fb70d5b',
 '9d1e4af323354cf180970e0b25099808',
 '880bc461d8d24192aa6361fff6dbdcfe',
 '67043300d2d34967bcd91cd738f81534',
 '44524dcf245448f6b6934b39cafdd5fb',
 'bd4734f1e2ad4ee7823f7f1f1422aaa9',
 '4474d879c9974045b51a96f9e8953034',
 'eb3fe3926f174770a680692d2ddfff54',
 'f2172b811a164884ac7f1622862812cf',
 '46d4e75344684e27b050b6121ad6a223',
 'bee45ce46560435ba2ef75ccdc45a7df',
 'ce52ad8e4ddd4aaea5300aa997b518b1',
 'bb66075a28fa44a2beabd4613e79d643',
 'd819fc3e85ab4eb7a3e17785d9d64e48',
 '01d5964442f94973a50183c3b3d89e2d',
 '5bcf29fcbe12417192e1b3fcab712637']

In [10]:
retriever = vstore.as_retriever(
    search_type="similarity",  # or "mmr" for diversity
    search_kwargs={"k": 3}     # top 3 most relevant chunks
)

## Retrival

In [11]:
query = "What is discussed in the PDF about Cyberpunk 2077 sales?"

# Convert query into an embedding
query_embedding = embeddings.embed_query(query)

# Perform similarity search directly on AstraDB
results = vstore.similarity_search(query, k=3)

print(f"Top {len(results)} retrieved chunks:\n")
for i, doc in enumerate(results, 1):
    print(f"--- Chunk {i} ---")
    print(doc.page_content[:300], "...\n")  # preview text


Top 3 retrieved chunks:

--- Chunk 1 ---
AAA Games Information
1. Cyberpunk 2077 - Company: CD Projekt Red - Release Year: 2020 - Sales: Over 25 million copies
sold by 2023 - Revenue: Estimated over $750 million from initial release and expansions - Notes:
Despite a troubled launch with bugs, post-launch patches and the "Phantom Liberty" e ...

--- Chunk 2 ---
Math and statistics are fundamental for data science and AI as they draw meaningful insights from complex datasets.



Topics to Learn in Statistics

Descriptive statistics

Inferential statistics

Basic plot in statistics

Measure of central tendency

Types of distributions

Central limit theorem

 ...

--- Chunk 3 ---
Generative AI

Roadmap 2024



From Basics to Advanced

Here's a step-by-step guide designed for absolute beginners looking to acquire skills in Generative AI. 

Different Positions or Different Levels:  

Developer Level 1 or Beginner Level 

Developer Level 2 or Senior Level

 Researcher Leve



 ...



## Generation

In [12]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template(
    """You are a helpful assistant.
Use the following context to answer the question:

Context:
{context}

Question:
{question}"""
)

In [13]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model = "gemini-2.5-flash-lite", api_key = GOOGLE_API_KEY)

In [14]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [15]:
response = chain.invoke("can you tell me the roadmap of generative ai?")

In [16]:
print("Answer:", response)

Answer: The roadmap for Generative AI, as outlined in the document, is a step-by-step guide for beginners to acquire skills. It's broken down into several sections:

*   **Prerequisite:** This section likely covers foundational knowledge needed before diving into Generative AI.
*   **Fundamentals:** This section will cover the basic concepts of Generative AI.
*   **Core Generative Models:** This section delves into the main types of generative models, including:
    *   **Generative Image Models:** Such as GANs (Generative Adversarial Networks), Variational Autoencoders (VAEs), and Diffusion Models (like Stable Diffusion).
    *   **Generative Language Models (LLMs):** Which are focused on natural language processing. Milestone LLM models mentioned include BERT, GPT, XLM, T5, Megatron, and M2M-100. Specific OpenAI LLM models highlighted are GPT-4, GPT-4 Turbo, GPT-3.5, and DALL·E.
*   **Developing Applications Powered by LLMs:** This section focuses on building applications that utiliz